<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Pandas for Exploratory Data Analysis: Joining Data

_Authors: Kevin Markham (DC)_

In [1]:
import pandas as pd

In [2]:
DATA_DIR = "../assets/data"

## Concatenating DataFrames

`pd.concat` sticks DataFrames together, either on top of each other or next to each other.

In [3]:
# Load movie titles
movie_cols = ["movie_id", "title"]
movies_path = f"{DATA_DIR}/movies.tbl"
movies = pd.read_csv(
    movies_path,
    sep="|",
    header=None,
    names=movie_cols,
    usecols=[0, 1],
    encoding="latin-1",
)
movies.head()

movie_id              title
0         1   Toy Story (1995)
1         2   GoldenEye (1995)
2         3  Four Rooms (1995)
3         4  Get Shorty (1995)
4         5     Copycat (1995)

In [4]:
movies.tail()

movie_id                                      title
1677      1678                          Mat' i syn (1997)
1678      1679                           B. Monkey (1998)
1679      1680                       Sliding Doors (1998)
1680      1681                        You So Crazy (1994)
1681      1682  Scream of Stone (Schrei aus Stein) (1991)

In [5]:
# Define a DataFrame with some additional movie titles
movie_ids = [1683, 1684, 1685, 1686, 1687]

more_movies = pd.DataFrame(
    {
        "movie_id": movie_ids,
        "title": [
            "Avengers: Endgame",
            "Captain Marvel",
            "Us",
            "How to Train Your Dragon: The Hidden World",
            "Shazam!",
        ],
    },
    index=[num - 1 for num in movie_ids],
)
more_movies.head()

movie_id                                       title
1682      1683                           Avengers: Endgame
1683      1684                              Captain Marvel
1684      1685                                          Us
1685      1686  How to Train Your Dragon: The Hidden World
1686      1687                                     Shazam!

*Stick the DataFrames on top of each other*

In [ ]:
# Define a DataFrame with running time information for some movies
movie_ids = range(2, 6)

movie_times = pd.DataFrame(
    {"movie_id": movie_ids, "running_mins": [130, 98, 105, 123],},
    index=[num - 1 for num in movie_ids],
)
movie_times

*Stick the DataFrames next to each other*

**Note:** As illustrated above, `pd.concat` uses the column names or row index values to align DataFrames, rather than column or row positions.

## Joining (Merging) DataFrames

### Movie Reviews Example

"Tidy data" is data that is organized as follows:

- One variable per column
- One observation per row
- One type of observational unit per table

For instance, our table of movies might exist alongside a table of movie reviews.

In [ ]:
# Load movie ratings
rating_cols = ["user_id", "movie_id", "rating", "timestamp"]
ratings_path = f"{DATA_DIR}/movie_ratings.tsv"
ratings = (
    pd.read_csv(ratings_path, sep="\t", header=None, names=rating_cols)
    .reset_index()
    .rename(columns={"index": "review_id"})
)
ratings.head()

Now if we want to know the title associated with a particular rating, we match up the rows of the two tables by `movie_id`:

![](../assets/images/inner_join.png)

SQL calls this "matching up" operation a **join**. Pandas has a method called "join," but it also has a more flexible method `.merge()`.

> **Side note:** Excel uses `VLOOKUP` for this kind of operation.

*Merge 'movies' and 'ratings' (inner join on 'movie_id')*

Equivalent SQL:

```sql
SELECT *
FROM movies
JOIN ratings
ON movie_id;
```

In [ ]:
print(movies.shape)
print(ratings.shape)
print(movie_ratings.shape)

Datasets are often stored in tidy format because that format is space-efficient, flexible, and conceptually clean. However, it may require you to do multiple joins to get all of the information you want for your model in one place.

### Selecting Columns to Merge On

*Define another DataFrame with additional movie reviews*

In [ ]:
more_ratings = pd.DataFrame(
    {
        "user_id": [712, 214, 86, 1, 14],
        "movie_id": [24, 1012, 1502, 1682, 1684],
        "rating": [3, 1, 2, 2, 5],
        "timestamp": [870069583, 870069683, 870069701, 870069824, 870070800],
    }
)

By default, `merge` matches up rows that have the same values for columns that are common between the two tables.

For instance, `ratings` and `more_ratings` have four columns in common, so `.merge()` will try (and fail) to find rows from the two tables that match on all of those columns.

*Join `more_ratings` to `ratings` on all columns*

Equivalent SQL:

```sql
SELECT *
FROM ratings
JOIN more_ratings
ON user_id, movie_id, rating, timestamp;
```

You can override this default behavior of `merge` by telling it exactly which columns to use. For instance, we might look for cases where two reviewers gave a movie the same rating.

*Join `more_ratings` to `ratings` on the "movie_id" column*

Equivalent SQL:

```sql
SELECT *
FROM ratings
JOIN more_ratings
ON movie_id, rating;
```

Notice that `merge` adds suffixes (here "\_x" and "\_y") to the names of the columns from `ratings` and `more_ratings`, respectively, that are common between `ratings` and `more_ratings` but were not used in the join.

You can also tell `merge` to match on columns with different names. For instance, suppose we wanted to join `movies` and `ratings`, but `movies` used "id" in place of "movie_id."

*Uncomment and run the cell below to try to merge `movies` to `ratings` after renaming "movie_id" in movies -- why does it fail?*

In [ ]:
# movies.rename(columns={"movie_id": "id"}).merge(ratings)

*Try again, but tell `merge` what columns to join on for each DataFrame*

Equivalent SQL:

```sql
SELECT *
FROM movies a
JOIN ratings b
ON a.id = b.movie_id;
```

You can also `merge` to join on the indices. For instance, suppose `movie_id` were the index of `movies`.

*Join `ratings` to `movies` with "movie_id" in the index of `movie`*

### Join Types

You might have noticed that when we used `merge` above, we dropped rows from the original DataFrames that weren't matched. That kind of join is called an "inner join." You can also do an "outer join", where rows that don't have a match are returned with missing values for the fields that would have come from the other table if there had been a match.

![](../assets/images/outer_join.png)

*Do an outer join of `more_ratings` to `movies`*

Equivalent SQL:

```sql
SELECT *
FROM movies
FULL OUTER JOIN more_ratings
ON movie_id;
```

You can also do a "left merge" that returns all rows from the "left" DataFrame only regardless of whether they have a match.

![](../assets/images/left_join.png)

*Join `more_ratings` to `movies`, keeping all rows from `movies` even if they don't have a match*

Equivalent SQL:

```sql
SELECT *
FROM movies
LEFT JOIN more_ratings
ON movie_id;
```

And you can do a "right merge" that returns all rows from the "right" DataFrame only regardless of whether they have a match.

![](../assets/images/right_join.png)

*Join `more_ratings` to `movies`, keeping all rows from `more_ratings` even if they don't have a match*

Equivalent SQL:

```sql
SELECT *
FROM movies
RIGHT JOIN more_ratings
ON movie_id;
```

![](../assets/images/join_types.png)

**Exercise**

*Time:* 2 min\
*Format:* Individual\
*Post answers:* Yes

In [ ]:
# Create some stock price data
openprice = pd.DataFrame(
    {
        "Symbol": ["AAPL", "DHR", "DAL", "AMZN"],
        "OpenPrice": [217.51, 96.54, 51.45, 1703.34],
    }
)
wkhigh = pd.DataFrame(
    {
        "Symbol": ["DAL", "AMZN", "AAPL", "DHR"],
        "52wkHigh": [60.79, 2050.49, 233.47, 110.11],
    }
)
stockname = pd.DataFrame(
    {
        "Symbol": ["AMZN", "DHR", "DAL", "AAPL"],
        "Name": ["Amazon", "Danaher", "Delta Airlines", "Apple"],
    }
)

- Inner join the `openprice` and `wkhigh` DataFrames.

- Inner join the `openprice` and `stockname` DataFrames.

- Inner join all three DataFrames.

$\blacksquare$

### Keys

In a traditional relational database, each table has a "primary key" that uniquely identifies each row of the table. For instance, our `movies` DataFrame has the `movie_id` column that uniquely identifies each movie.

![](../assets/images/primary_key.png)

`ratings` also has a column `movie_id`, but it is not a primary key for `ratings` -- it has repeated values. Its purpose in `ratings` is to allow us to join `ratings` with `movies`. Because it refers to the primary key of another table, it is called a *foreign key* in `ratings`.

![](../assets/images/foreign_key.png)

### Sales Data Example

Here are the data dictionaries we'll be using for the following exercise:

<ul>
    <li><a href="https://www.sqldatadictionary.com/AdventureWorks2014/Production.Product.html">Production.Product</a></li>
    <li><a href="https://www.sqldatadictionary.com/AdventureWorks2014/Production.ProductSubcategory.html">Production.ProductSubcategory</a></li>
    <li><a href="https://www.sqldatadictionary.com/AdventureWorks2014/Sales.SalesOrderHeader.html">Sales.SalesOrderHeader</a></li>
    <li><a href="https://www.sqldatadictionary.com/AdventureWorks2014/Sales.SalesOrderDetail.html">Sales.SalesOrderDetail</a></li>
</ul>

In [ ]:
# Load the data
product = pd.read_csv(f"{DATA_DIR}/Production.Product.csv", sep="\t")
product_subcategory = pd.read_csv(
    f"{DATA_DIR}/Production.ProductSubcategory.csv", sep="\t"
)
sales_header = pd.read_csv(
    f"{DATA_DIR}/Sales.SalesOrderHeader.csv", sep="\t", nrows=1000
)
sales_detail = pd.read_csv(
    f"{DATA_DIR}/Sales.SalesOrderDetail.csv", sep="\t", nrows=1000
)

The [Production.ProductSubcategory data dictionary](https://www.sqldatadictionary.com/AdventureWorks2014/Production.ProductSubcategory.html) reports a "primary key" `ProductSubcategoryID`.

The [Production.Product data dictionary](https://www.sqldatadictionary.com/AdventureWorks2014/Production.Product.html) characterizes `ProductSubcategoryID` as a "foreign key" because it corresponds to the primary key of `Production.ProductSubcategory`.

*Join the Production.Product and Production.ProductSubcategory on `ProductSubcategoryID`*

Equivalent SQL (except for the "\_sub" suffix):

```sql
SELECT *
FROM product
INNER JOIN product_subcategory
ON ProductSubcategoryID;
```

**Exercise**

*Time:* 5 mins\
*Format:* Pairs\
*Post answers:* Yes

- Left join `Sales.SalesOrderDetail` (right) to `Sales.SalesOrderHeader` (left) on the relevant primary/foreign key. (Use the data dictionaries to identify the key. There seems to be a mistake in the `Sales.SalesOrderDetail` data dictionary: "SalesOrderId" is only a foreign key in that table and not a primary key.)

- Left join `Sales.SalesOrderDetail` (right) to `Sales.SalesOrderHeader` (left) on the relevant primary/foreign key, and left join `Production.Product` to the result on the relevant primary/foreign key.

$\blacksquare$

## Summary

- "Tidy data" is organized as follows:
    - One variable per column
    - One observation per row
    - One type of observational unit per table
- Storing data in tidy format saves space and is conceptually nice, but it might mean that you have to use multiple tables to answer a given question.
- The Pandas `.merge()` method allows you to join tables on common index and column values.